# Restricted Boltzmann Machine Defintion

In [1]:
# TODO g = 1 + ap of g = ap

In [1]:
# Import PyTorch library
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import nn, flatten, device

device = torch.device('cpu')
if torch.cuda.is_available():
    device = torch.device('cuda')

In [2]:
# https://github.com/khanhnamle1994/MetaRec/blob/b5e36cb579a88b32cdfb728f35f645d76b24ad95/Boltzmann-Machines-Experiments/RBM-CF-PyTorch/rbm.py#L23
# Create the Restricted Boltzmann Machine architecture
class network(nn.Module):
    def __init__(self, x ):
        super().__init__()
        
    #use 4 layers and fc layer
        self.conv1 = nn.Conv1d(x, 256, kernel_size=1)
        self.conv2 = nn.Conv1d(256, 1024, kernel_size=1)
        self.conv3 = nn.Conv1d(1024, 512, kernel_size=1)
        self.conv4 = nn.Conv1d(512, 64, kernel_size=1)
        self.fc = nn.Linear(64*7*7, x)

        self.dropout = nn.Dropout(0.25)
        if torch.cuda.is_available():
            self.device = device("cuda")

    def forward(self, x):
        if torch.cuda.is_available():
            x = x.to(self.device)

        x = F.relu(self.conv1(x))

        # x = F.max_pool1d(x, 2)
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = flatten(x, 1)

        x = self.dropout(x)
        x = self.fc(x)
        output = F.softmax(x, dim=1)
        return output
    
    def lr(self):
        return 0.01



    def train_model(self, v0, vk, ph0, phk):
        """
        Perform contrastive divergence algorithm to optimize the weights that minimize the energy
        This maximizes the log-likelihood of the model
        """

        ph0_K = torch.stack([ph0 for _ in range(self.K)])
        phk_K = torch.stack([phk for _ in range(self.K)])

        pos = torch.bmm(torch.transpose(v0, 1, 2).cpu(), ph0_K.cpu())
        neg = torch.bmm(torch.transpose(vk, 1, 2).cpu(), phk_K.cpu())
        if torch.cuda.is_available():
            pos = pos.cuda()
            neg = neg.cuda()

        w_extra = torch.transpose(pos - neg, 1, 2)
        v_extra = torch.sum((v0 - vk), 1)
        h_extra = torch.sum((ph0 - phk), 0)

        # if self.i % 45 == 0:
            # print(torch.max(w_extra), torch.max(v_extra), torch.max(h_extra), flush=True)

        # Approximate the gradients with the CD algorithm
        # TODO learning rate toevoegen
        self.W += self.lr() * w_extra

        # Add (difference, 0) for the tensor of 2 dimensions
        self.v_bias += self.lr() * v_extra.unsqueeze(1)
        self.h_bias += self.lr() * h_extra
        self.i += 1

In [3]:
cuda = torch.device('cuda')

# General Imports

In [4]:
import numpy as np
import pickle as pickle
import pandas as pd
import scipy
import sklearn
import gzip
import json
from tqdm import tqdm
import os
from collections import Counter
from datetime import datetime
import math
tqdm.pandas() #for progres_apply etc.

# Reading

In [5]:
with open("stijn-data/item_dct.p", "rb") as f:
    item_dict = pickle.load(f)


In [6]:
def parse_csv(filename):
    df_string = pd.read_csv(filename, index_col=0)
    df = df_string.loc[:,:]
    df["item_id"] = df["item_id"].apply(eval)
    df["playtime_forever"] = df["playtime_forever"].apply(eval)
    df["playtime_2weeks"] = df["playtime_2weeks"].apply(eval)
    return df

In [7]:
train0 = parse_csv("stijn-data/train_split_0.csv")
test0 = parse_csv("stijn-data/test_split_0.csv")

In [8]:
train0["item_id"].map(len).describe()

count    65701.000000
mean        50.869926
std         64.529794
min          2.000000
25%         12.000000
50%         31.000000
75%         64.000000
max        716.000000
Name: item_id, dtype: float64

In [9]:
train0.iloc[100,:]

user_id                                                           100
item_id             [221, 1698, 532, 318, 178, 101, 171, 576, 229,...
playtime_forever    [1254, 67, 1075, 597, 5195, 395, 212, 363, 574...
playtime_2weeks                     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Name: 100, dtype: object

# Sparse Matrix

Create Sparse Matrix

In [10]:
def score_playtime(playtime):
    
    if playtime < 120:
        # less than 2 hrs
        return 0
    elif playtime < 240:
        # less than 4 hrs
        return 1
    elif playtime < 600:
        # less than 10 hrs
        return 2
    elif playtime < 24*60:
        # less than 24 hrs
        return 3
    else:
        return 4

#Create scipy csr matrix
def get_sparse_matrix(df):
    shape = (df['user_id'].max() + 1, max(item_dict.values()) + 1)
    
    user_ids = []
    item_ids = []
    values = []
    for idx, row in df.iterrows():
        items = row['item_id']
        user = row['user_id']
        score = row["playtime_forever"] + 2* row["playtime_2weeks"]
        
        
    
        # recommended = row['recommended']
        user_ids.extend([user] * len(items))
        item_ids.extend(items)
        values.extend([score_playtime(score[i]) for i in range(len(items))])
    # create csr matrix
    # values = np.ones(len(user_ids))
    matrix = scipy.sparse.csr_matrix((values, (user_ids, item_ids)), shape=shape, dtype=np.int32)
    return matrix


In [11]:
print('hey')
test_matrix = get_sparse_matrix(test0)

train_matrix = get_sparse_matrix(train0)
train_matrix


hey


<65701x10269 sparse matrix of type '<class 'numpy.intc'>'
	with 3342205 stored elements in Compressed Sparse Row format>

In [12]:
torch.manual_seed(0)

Train model

In [13]:
def score_model(rbm, batch_size, train_matrix, test_matrix):
    test_recon_error = 0  # RMSE reconstruction error initialized to 0 at the beginning of training
    s = 0  # a counter (float type) 
    # for loop - go through every single user
    for id_user in range(0, train_matrix.shape[0] - batch_size, batch_size):
        v = train_matrix[id_user:id_user + batch_size]  # training set inputs are used to activate neurons of my RBM
        vt = test_matrix[id_user:id_user + batch_size]  # target
        v = convert_sparse_matrix_to_sparse_tensor(v)
        vt = convert_sparse_matrix_to_sparse_tensor(vt)

        v = v.to_dense()
        vt = vt.to_dense()
        v = v.sub(1)
        vt = vt.sub(1)
        
        if torch.cuda.is_available():
            v = v.cuda()
            vt = vt.cuda()

        if len(vt[vt > -1]) > 0:
            v = rbm(v)
            
            # Update test RMSE reconstruction error
            test_recon_error += torch.mean((vt[vt > -1] - v[vt > -1])**2) * len(vt > -1) 
            s += len(vt > -1)

    return torch.sqrt(test_recon_error / s)


# https://stackoverflow.com/questions/40896157/scipy-sparse-csr-matrix-to-tensorflow-sparsetensor-mini-batch-gradient-descent


def convert_sparse_matrix_to_sparse_tensor(X, k=5):
    coo = X.tocoo()

    values = coo.data
    indices = np.vstack((coo.row, coo.col))
    i = torch.LongTensor(indices)
    v = torch.DoubleTensor(values)
    tensor_list = []

    for index in range(k):
        value = index
        yeet = torch.where(v == value, 2., 1.)
        shape = coo.shape
        tensor = torch.sparse.DoubleTensor(i, yeet, torch.Size(shape)) 
        if torch.cuda.is_available():
            tensor = tensor.cuda()

        tensor_list.append(tensor)

    tensor = torch.stack(tensor_list) 
    return tensor
def create_rbm(train_matrix, test_matrix, n_hidden, batch_size, epochs, rbm=None, k=5):
    n_vis = train_matrix.shape[1]
    train_errors = []
    test_errors = []
    if rbm is None:
        rbm = (n_vis, n_hidden, k, batch_size)
    optim = torch.optim.SGD(rbm.parameters(), lr=0.02, momentum=0.9)
    print("start training")
    for epoch in tqdm(range(epochs)):
        rbm.train()
        train_recon_error = 0  # RMSE reconstruction error initialized to 0 at the beginning of training
        s = 0
        
        for user_id in range(0, train_matrix.shape[0] - batch_size, batch_size):
            training_sample = train_matrix[user_id : user_id + batch_size]
            v0 = convert_sparse_matrix_to_sparse_tensor(training_sample)

            v0 = v0.to_dense()
            v0 = v0.sub(1)
            

            vk = rbm(v0)


                   
            
            loss = torch.sqrt(torch.mean((v0[v0 > -1] - vk[v0 > -1])**2))
            
            loss.backward()
            optim.step()
            optim.zero_grad()
            
            train_recon_error +=loss
            
            s += len(v0 > -1)
            
        train_errors.append(train_recon_error / s)

        # print('calculating test scores')
        rbm.eval()
        test_errors.append(score_model(rbm, batch_size, train_matrix, test_matrix))

    import matplotlib.pyplot as plt
    # Plot the RMSE reconstruction error with respect to increasing number of epochs
    plt.plot(torch.Tensor(train_errors, device='cpu'), label="train")
    plt.plot(torch.Tensor(test_errors, device='cpu'), label="test")
    plt.ylabel('Error')
    plt.xlabel('Epoch')
    plt.legend()
    plt.savefig(f'aussies-{n_hidden}-{batch_size}-{epochs}.jpg')

    return rbm

# Evaluate the RBM on test set
# test_recon_error = score_model(rbm)
# print("Final error", test_recon_error)


## HR / Recall / NDCG Function Definitions

### Vanilla Recommendations

In [14]:
def compute_hr(train_matrix, test_matrix, rbm, k=10, batch_size=100):
    s = 0  # a counter (float type) 
    hitrates = []
    recall = []
    nDCG = []
    # for loop - go through every single user
    for id_user in range(0, train_matrix.shape[0]): # - batch_size, batch_size):
        v = train_matrix[id_user]  # training set inputs are used to activate neurons of my RBM
        vt = test_matrix[id_user]  # target

        target_data = vt.data
        target_index = vt.indices
        target_recommendations = target_index[target_data == 1]
        # print(target_test)

        v = v.todense()

        # v = v - 1
        v = torch.Tensor(v)
        if torch.cuda.is_available():
            v = v.cuda()
        
        if len(target_recommendations) > 0: # check that target contains recommendations (only needed for aussies)
            _, h = rbm.sample_h(v)
            recommended, _ = rbm.sample_v(h)

            # all recommendations
            _, indices =  torch.topk(recommended[v < 1], k)
            recommendations = indices.cpu().tolist()

            counter = 0
            total = min(len(target_recommendations), k)
            for target in target_recommendations:
                if target in recommendations:
                    counter += 1
            # counter = len(recommendations)

            recall.append(counter / total)
            hitrates.append(min(1, counter))

            # nDCG
            idcg = np.sum([1 / np.log2(i+2) for i in range(min(k, len(target_recommendations)))])
            dcg = 0
            for i, r in enumerate(recommendations):
                if r in target_recommendations:
                    dcg += 1 / np.log2(i+2)

            nDCG.append(dcg / idcg) 

    return hitrates, recall, nDCG


In [ ]:
rbm10 = create_rbm(train_matrix, test_matrix, 1000, 10000, 2)

  0%|          | 0/2 [00:00<?, ?it/s]

start training


In [17]:
# rbm20 =create_rbm(train_matrix, test_matrix, 1000, 10000, 10, rbm10)


### Popularity

In [18]:
def compute_hr2(pops, k=10, batch_size=100):
    s = 0  # a counter (float type) 
    hitrates = []
    recall = []
    nDCG = []
    # idcg = np.sum([1 / np.log2(i+2) for i in range(k)])
    # for loop - go through every single user
    for id_user in range(0, train_matrix.shape[0]): # - batch_size, batch_size):
        v = train_matrix[id_user]  # training set inputs are used to activate neurons of my RBM
        vt = test_matrix[id_user]  # target

        target_data = vt.data
        target_index = vt.indices
        target_recommendations = target_index[target_data == 2]
        # print(target_test)

        
        if len(target_recommendations) > 0: # check that target contains recommendations (only needed for aussies)
            # _, h = rbm.sample_h(v)
            # recommended, _ = rbm.sample_v(h)
            # 
            # # all recommendations
            # _, indices =  torch.topk(recommended[v < 0], k)
            recommendations = pops[:k]

            counter = 0
            total = min(len(target_recommendations), k)
            for target in target_recommendations:
                if target in recommendations:
                    counter += 1
            # counter = len(recommendations)

            recall.append(counter / total)
            hitrates.append(min(1, counter))

            # nDCG
            idcg = np.sum([1 / np.log2(i+2) for i in range(min(k, len(target_recommendations)))])
            counter = 0
            dcg = 0
            for i, r in enumerate(recommendations):
                if r in target_recommendations:
                    dcg += 1 / np.log2(i+2)

            nDCG.append(dcg / idcg) 

    return hitrates, recall, nDCG



### RBM + Popularity

In [19]:
def compute_hr3(rbm, popularity_dict, k=10):
    s = 0  # a counter (float type) 
    hitrates = []
    recall = []
    nDCG = []
    idcg = np.sum([1 / np.log2(i+2) for i in range(k)])
    pop = torch.FloatTensor(popularity_dict)
    if torch.cuda.is_available():
        pop = pop.cuda()

    # for loop - go through every single user
    for id_user in range(0, train_matrix.shape[0]): # - batch_size, batch_size):
        v = train_matrix[id_user]  # training set inputs are used to activate neurons of my RBM
        vt = test_matrix[id_user]  # target

        target_data = vt.data
        target_index = vt.indices
        target_recommendations = target_index[target_data == 2]
        # print(target_test)

        v = v.todense()

        v = v - 1
        v = torch.Tensor(v)
        if torch.cuda.is_available():
            v = v.cuda()
        
        if len(target_recommendations) > 0: # check that target contains recommendations (only needed for aussies)
            _, h = rbm.sample_h(v)
            recommended, _ = rbm.sample_v(h)

            # all recommendations
            # multiply recommendations by the games popularity
            # print(recommended)
            recommended = torch.mul(recommended, pop)
            # print(recommended)
            _, indices =  torch.topk(recommended[v < 0], k)
            # recommendations = torch.tensor(indices, device='cpu').tolist()
            recommendations = indices.cpu().tolist()

            counter = 0
            total = min(len(target_recommendations), k)
            for target in target_recommendations:
                if target in recommendations:
                    counter += 1
            # counter = len(recommendations)

            recall.append(counter / total)
            hitrates.append(min(1, counter))

            # nDCG
            counter = 0
            dcg = 0
            for i, r in enumerate(recommendations):
                if r in target_recommendations:
                    dcg += 1 / np.log2(i+2)

            nDCG.append(dcg / idcg) 

    return hitrates, recall, nDCG

## HR / Recall / NDCG

In [ ]:
def evaluate_rbm(rbm):
    print("Vanilla RBM")
    hr, r, ndcg = compute_hr(train_matrix, test_matrix, rbm)
    # print(hr, r, ndcg)
    print("hr", np.average(hr))
    print("recall", np.average(r))
    print("ndcg", np.average(ndcg))

    # print("popularity incorporated")
    # hr, r, ndcg = compute_hr3(rbm, value_list)
    # print("hr", np.average(hr))
    # print("recall", np.average(r))
    # print("ndcg", np.average(ndcg))
    

In [21]:
evaluate_rbm(rbm10)

NameError: name 'rbm10' is not defined

In [ ]:
rbm100 = create_rbm(train_matrix, test_matrix, 1024, 10240, 100)
evaluate_rbm(rbm100)

In [ ]:
rbm10 = create_rbm(train_matrix, test_matrix, 1024, 10240, 10)
rbm20 = create_rbm(train_matrix, test_matrix, 1024, 10240, 20)

In [ ]:
print("10 epochs")
hr, r, ndcg = compute_hr(train_matrix, test_matrix, rbm10)
print("hr", np.average(hr))
print("recall", np.average(r))
print("ndcg", np.average(ndcg))

print("20 epochs")
hr, r, ndcg = compute_hr(train_matrix, test_matrix,rbm20)
print("hr", np.average(hr))
print("recall", np.average(r))
print("ndcg", np.average(ndcg))

print("popularity")
hr,r ,ndcg = compute_hr2(popularity.index)
print("hr", np.average(hr))
print("recall", np.average(r))
print("ndcg", np.average(ndcg))

print("popularity incorporated")
hr, r, ndcg = compute_hr3(rbm10, value_list)
print("hr", np.average(hr))
print("recall", np.average(r))
print("ndcg", np.average(ndcg))


In [ ]:
def recommend(rbm, v, vt, k, p=True):
    target_data = vt.data
    target_index = vt.indices
    target_recommendations = target_index[target_data == 1]
    v = v.todense()
    # v = v - 1
    v = torch.Tensor(v)
    if torch.cuda.is_available():
        v = v.cuda()
    
    _, h = rbm.sample_h(v)
    recommended, _ = rbm.sample_v(h)

    # all recommendations
    values, indices =  torch.topk(recommended[v < 1], k)
    recommendations = indices.cpu().tolist()

    if p:
        # print('20', recommended[0][20])
        # print('21', recommended[0][21])
        print("average value", torch.mean(recommended[0]))

    found = True
    for r in recommendations:
        if r in target_recommendations:
            if p:
                print("HIT")
            found = True
            break

    if found and p:
        print("values", values)
        print("recommended", recommendations)
        print("real", target_recommendations)
        print("len real", len(target_recommendations))

    
    
    return recommendations

user = 100
# print("train", train_matrix[user])
# print("test", test_matrix[user])


In [ ]:

print("EPOCHS 10")
recommend(rbm10, train_matrix[user], test_matrix[user], 10)
print('---' * 10)
print("EPOCHS 100")
recommend(rbm100, train_matrix[user], test_matrix[user], 10)

In [ ]:
import gc

gc.collect()

torch.cuda.empty_cache()

In [ ]:
rbm2000 = create_rbm(train_matrix, test_matrix, 100, 10000, 2000)

In [ ]:

recommend(rbm2000, train_matrix[user], test_matrix[user], 10)

In [ ]:
print("2000 epochs")
hr, r, ndcg = compute_hr(train_matrix, test_matrix, rbm2000)
print("hr", np.average(hr))
print("recall", np.average(r))
print("ndcg", np.average(ndcg))

In [ ]:
# hrs = []
# rs = []
# for i in range(10):
#     rbm = create_rbm(user_reviews_df_exploded['item_id_int'].max() + 1, 1024, 10240, i)
#     hr, r = compute_hr(rbm)
#     hrs.append(np.average(hr))
#     rs.append(np.average(r))

In [ ]:
# plt.clf()
# plt.plot(hrs, label='HR')
# plt.plot(rs, label='Recall')
# plt.xlabel("Epochs")
# plt.legend()

In [ ]:
rbm1 = create_rbm(user_reviews_df_exploded['item_id_int'].max() + 1, 1024, 10240, 50)
hr, r, ndcg = compute_hr(rbm1)
print("hr", np.average(hr))
print("recall", np.average(r))
print("ndcg", np.average(r))

In [ ]:
rbm50 = create_rbm(user_reviews_df_exploded['item_id_int'].max() + 1, 1024, 10240, 50)
hr, r = compute_hr(rbm50)
print("hr", np.average(hr))
print("recall", np.average(r))

In [ ]:
print("EPOCHS 50")
recommend(rbm50, train_matrix[user], test_matrix[user], 10)

In [ ]:
rbm100 = create_rbm(user_reviews_df_exploded['item_id_int'].max() + 1, 1024, 10240, 100)
hr, r, ndcg = compute_hr(rbm100)
print("hr", np.average(hr))
print("recall", np.average(r))
print("ndcg", np.average(ndcg))

In [ ]:
all_recommendations = set()
for u in range(train_matrix.shape[0]):
    recommendations = recommend(rbm50, train_matrix[u], test_matrix[u], 10, False)
    all_recommendations.update(recommendations)

print(len(all_recommendations))

In [ ]:
all_recommendations = set()
for u in range(train_matrix.shape[0]):
    recommendations = recommend(rbm20, train_matrix[u], test_matrix[u], 10, False)
    all_recommendations.update(recommendations)

print(len(all_recommendations))

In [ ]:
all_recommendations = set()
for u in range(train_matrix.shape[0]):
    recommendations = recommend(rbm10, train_matrix[u], test_matrix[u], 10, False)
    all_recommendations.update(recommendations)

print(len(all_recommendations))

In [ ]:
print(train_matrix.shape[1])

In [ ]:
rbm500 = create_rbm(user_reviews_df_exploded['item_id_int'].max() + 1, 1024, 10240, 500)
hr, r = compute_hr(rbm50)
print("hr", np.average(hr))
print("recall", np.average(r))

all_recommendations = set()
for u in range(train_matrix.shape[0]):
    recommendations = recommend(rbm500, train_matrix[u], test_matrix[u], 10, False)
    all_recommendations.update(recommendations)

print(len(all_recommendations))

In [ ]:
hr, r = compute_hr(rbm500)
print("hr", np.average(hr))
print("recall", np.average(r))

In [ ]:
rbm2000 = create_rbm(user_reviews_df_exploded['item_id_int'].max() + 1, 1024, 10240, 2000)
hr, r, ndcg = compute_hr(rbm2000)
print("hr", np.average(hr))
print("recall", np.average(r))
print("ndcg", np.average(ndcg))

all_recommendations = set()
for u in range(train_matrix.shape[0]):
    recommendations = recommend(rbm2000, train_matrix[u], test_matrix[u], 10, u == train_matrix.shape[0] - 1)
    all_recommendations.update(recommendations)

print(len(all_recommendations))

## Hyperparam searching

In [ ]:
epochs = [100]
hidden = [100, 1000, 5000, 10000]
rbms = []
results = {}

for epoch in epochs:
    for n_hidden in hidden:
        rbm = create_rbm(train_matrix, test_matrix, n_hidden, 10240, epoch)
        rbms.append(rbm)
        hr, r, ndcg = compute_hr(train_matrix, test_matrix, rbm)
        results[f"{epoch}-{n_hidden}"] = (np.average(hr), np.average(r), np.average(ndcg))

results

In [ ]:
for i, rbm in enumerate(rbms):
    hr, r, ndcg = compute_hr(train_matrix, test_matrix, rbm)
    results[f"{100}-{hidden[i]}"] = (np.average(hr), np.average(r), np.average(ndcg))
    
results

In [ ]:
import pprint
pprint.pprint(results)

In [ ]:
# torch.save(rbm.state_dict(), "./network")

In [ ]:
# rbm = RBM(n_vis, n_hidden)
# rbm.load_state_dict(torch.load("./network"))
# rbm.eval()